## CIFAR-10 Retrained vs Redistributed Accuracy Table

### All Seed

In [ ]:
# %%
import os
import numpy as np
import pandas as pd
from lib.utils import model_results_from_npz  # Use your existing function

# %%
# ----- Config -----
NUM_CLASSES = 10
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]  # example seeds
EPOCHS = 50
CLASSES = ["airplane","automobile","bird","cat","deer",
           "dog","frog","horse","ship","truck"]

DIR_RETRAIN = "../results/cifar10/"
DIR_REDIS   = "../results/redis_cifar10/"

# %%
# ----- Collect results per removed class and per seed -----
all_results = {removed_idx: [] for removed_idx in range(NUM_CLASSES)}

for seed in SEEDS:
    print(f"Processing seed {seed}...")
    for removed_idx in range(NUM_CLASSES):
        removed_name = CLASSES[removed_idx]

        retrain_path = os.path.join(DIR_RETRAIN, f"cifar_resnet_s{seed}_e{EPOCHS}_r{removed_idx}.npz")
        redis_path   = os.path.join(DIR_REDIS,   f"cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_idx}.npz")

        if not (os.path.exists(retrain_path) and os.path.exists(redis_path)):
            print(f"  Skipping missing class {removed_name}")
            continue

        # Load models
        model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
        model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

        # Forget and retain accuracy
        forget_retrain = model_retrain.accuracy[removed_idx]
        retain_retrain = np.mean([model_retrain.accuracy[i] for i in range(NUM_CLASSES) if i != removed_idx])

        forget_redis = model_redis.accuracy[removed_idx]
        retain_redis = np.mean([model_redis.accuracy[i] for i in range(NUM_CLASSES) if i != removed_idx])

        all_results[removed_idx].append({
            "forget_retrain": forget_retrain,
            "retain_retrain": retain_retrain,
            "forget_redis": forget_redis,
            "retain_redis": retain_redis,
        })

# %%
# ----- Average results across seeds -----
rows = []
for removed_idx in range(NUM_CLASSES):
    removed_name = CLASSES[removed_idx]
    seed_results = all_results[removed_idx]

    if not seed_results:
        continue

    avg_forget_retrain = np.mean([r["forget_retrain"] for r in seed_results])
    avg_retain_retrain = np.mean([r["retain_retrain"] for r in seed_results])
    avg_forget_redis  = np.mean([r["forget_redis"]  for r in seed_results])
    avg_retain_redis  = np.mean([r["retain_redis"]  for r in seed_results])

    rows.append({
        "removed_class": removed_idx,
        "removed_name": removed_name,
        "forget_retrain": avg_forget_retrain,
        "retain_retrain": avg_retain_retrain,
        "forget_redis": avg_forget_redis,
        "retain_redis": avg_retain_redis,
    })

# %%
# ----- Build DataFrame -----
df_avg = pd.DataFrame(rows)
print(df_avg)

# Optionally save:
df_avg.to_csv("../analytics/CIFAR10/accuracy/redis_vs_retrain_accuracy_all_seed.csv", index=False)


Processing seed 42...
Processing seed 602...
Processing seed 311...
Processing seed 637...
Processing seed 800...
Processing seed 543...
Processing seed 969...
Processing seed 122...
Processing seed 336...
Processing seed 93...
   removed_class removed_name  forget_retrain  retain_retrain  forget_redis  \
0              0     airplane             0.0        0.926900           0.0   
1              1   automobile             0.0        0.921367           0.0   
2              2         bird             0.0        0.932767           0.0   
3              3          cat             0.0        0.941689           0.0   
4              4         deer             0.0        0.924756           0.0   
5              5          dog             0.0        0.937500           0.0   
6              6         frog             0.0        0.925189           0.0   
7              7        horse             0.0        0.916989           0.0   
8              8         ship             0.0        0.923778

### One Seed

In [3]:
# %% [markdown]
# # CIFAR-10 Retrained vs Redistributed Accuracy Table

# %%
import os
import numpy as np
import pandas as pd

from lib.utils import model_results_from_npz  # Use your existing function

# %%
# ----- Config -----
NUM_CLASSES = 10
SEED = 969 
EPOCHS = 50
CLASSES = ["airplane","automobile","bird","cat","deer",
           "dog","frog","horse","ship","truck"]

DIR_RETRAIN = "../results/cifar10/"
DIR_REDIS   = "../results/redis_cifar10/"

# %%
# ----- Collect results per removed class -----
rows = []

for removed_idx in range(NUM_CLASSES):
    removed_name = CLASSES[removed_idx]

    retrain_path = os.path.join(DIR_RETRAIN, f"cifar_resnet_s{SEED}_e{EPOCHS}_r{removed_idx}.npz")
    redis_path   = os.path.join(DIR_REDIS,   f"cifar_resnet_s{SEED}_e{EPOCHS}_rd{removed_idx}.npz")

    if not (os.path.exists(retrain_path) and os.path.exists(redis_path)):
        print(f"Skipping missing class {removed_name}")
        continue

    # Load models using your existing function
    model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
    model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

    # Forget and retain accuracy
    forget_retrain = model_retrain.accuracy[removed_idx]
    retain_retrain = np.mean([model_retrain.accuracy[i] for i in range(NUM_CLASSES) if i != removed_idx])

    forget_redis = model_redis.accuracy[removed_idx]
    retain_redis = np.mean([model_redis.accuracy[i] for i in range(NUM_CLASSES) if i != removed_idx])

    rows.append({
        "removed_class": removed_idx,
        "removed_name": removed_name,
        "forget_retrain": forget_retrain,
        "retain_retrain": retain_retrain,
        "forget_redis": forget_redis,
        "retain_redis": retain_redis,
    })

# %%
# ----- Build DataFrame -----
df = pd.DataFrame(rows)
df
# Optionally save as CSV:
df.to_csv(f"../analytics/CIFAR10/accuracy/redis_vs_retrain_accuracy_s{SEED}.csv", index=False)


# With Pretrained

In [1]:
# %% [markdown]
# # CIFAR-10 Original vs Retrained vs Redistributed Accuracy Table

# %%
import os
import numpy as np
import pandas as pd

from lib.utils import model_results_from_npz  # Your existing loader

# %%
# ----- Config -----
NUM_CLASSES = 10
SEED = 969 
EPOCHS = 50
CLASSES = ["airplane", "automobile", "bird", "cat", "deer",
           "dog", "frog", "horse", "ship", "truck"]

DIR_ORIG   = "../results/cifar10/"
DIR_RETRAIN = "../results/cifar10/"
DIR_REDIS   = "../results/redis_cifar10/"

# %%
# ----- Load original (unlearned) model -----
# Adjust filename if necessary
orig_path = os.path.join(DIR_ORIG, f"cifar_resnet_s{SEED}_e{EPOCHS}.npz")
if not os.path.exists(orig_path):
    orig_path = os.path.join(DIR_ORIG, f"cifar_resnet_s{SEED}_e{EPOCHS}_r-1.npz")

if not os.path.exists(orig_path):
    raise FileNotFoundError("Original model file not found. Please check filename convention.")

model_orig = model_results_from_npz(orig_path, NUM_CLASSES)

# %%
# ----- Collect results per removed class -----
rows = []

for removed_idx in range(NUM_CLASSES):
    removed_name = CLASSES[removed_idx]

    retrain_path = os.path.join(DIR_RETRAIN, f"cifar_resnet_s{SEED}_e{EPOCHS}_r{removed_idx}.npz")
    redis_path   = os.path.join(DIR_REDIS,   f"cifar_resnet_s{SEED}_e{EPOCHS}_rd{removed_idx}.npz")

    if not (os.path.exists(retrain_path) and os.path.exists(redis_path)):
        print(f"Skipping missing class {removed_name}")
        continue

    # Load models
    model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
    model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

    # Forget and retain accuracy for each
    def get_forget_retain(acc, removed_idx):
        forget = acc[removed_idx]
        retain = np.mean([acc[i] for i in range(NUM_CLASSES) if i != removed_idx])
        return forget, retain

    forget_orig, retain_orig = get_forget_retain(model_orig.accuracy, removed_idx)
    forget_retrain, retain_retrain = get_forget_retain(model_retrain.accuracy, removed_idx)
    forget_redis, retain_redis = get_forget_retain(model_redis.accuracy, removed_idx)

    rows.append({
        "removed_class": removed_idx,
        "removed_name": removed_name,

        "orig_forget": forget_orig,
        "orig_retain": retain_orig,

        "retrain_forget": forget_retrain,
        "retrain_retain": retain_retrain,

        "redis_forget": forget_redis,
        "redis_retain": retain_redis,
    })

# %%
# ----- Build DataFrame -----
df = pd.DataFrame(rows)
df = df[[
    "removed_class", "removed_name",
    "orig_retain", "orig_forget",
    "retrain_retain", "retrain_forget",
    "redis_retain", "redis_forget"
]]

df

# Optionally save as CSV
os.makedirs("../analytics/CIFAR10/accuracy", exist_ok=True)
df.to_csv(f"../analytics/CIFAR10/accuracy/redis_vs_retrain_vs_orig_accuracy_s{SEED}.csv", index=False)
print(f"Saved results for seed {SEED} ✅")


Saved results for seed 969 ✅
